In [35]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [17]:
from __future__ import print_function, absolute_import

import matplotlib.pyplot as plt

### 1) No-op

Step 1) Vanilla dircol traj. optimization.

In [3]:
from traj.dircol import make_real_dircol_mp

dircol, tree = make_real_dircol_mp(expmt="cartpole", seed=1776)

R = 10  # Cost on input "effort".
u = dircol.input()
dircol.AddRunningCost(R*u[0]**2)

Step 2) Simple supervised learning on (x, u) pairs, with no constraints on policy.

### 2) GPS

Step 1) dircol traj. optimization using Augmented Lagrangian

In [ ]:
def cost():
    pass
def Pi():
    pass
def lagrangian_t(x, u, theta, lam, rho=1): # TODO: figure out rho...
    global cost
    global Pi
    a = cost(x, u)       # Additive cost
    b = lam*(Pi(x)-u)    # Model L1 penalty?
    c = rho*(Pi(x)-u)**2 # Model L2 penalty?
    return a+b+c

In [47]:
import numpy as np

from traj.dircol import (
    apply_running_policy_cost,
    make_real_dircol_mp,
)

prog, tree = make_real_dircol_mp(expmt="cartpole", seed=1776)

R = 10  # Cost on input "effort".
u = prog.input()
prog.AddRunningCost(R*u[0]**2)

# Should we use an NN? or not.
def Pi_trivial(x_t):
    return (x_t[0]-2.)*(x_t[1]-2.)
def Pi_nn(x_t):
    from nn_system.NNSystemHelper import FC, MLP, NNInferenceHelper
    # net = FC(4).double()
    net = MLP(4).double()

    in_list = list(x_t)
    out_list = NNInferenceHelper(net, in_list)
    return out_list[0]

# Pi = Pi_trivial
Pi = Pi_nn

def mycost(Pi, x_t, u_t):
    lam, rho = 1, 1 # For now, set the dual variables to constants
    cost = lam * (Pi(x_t) - u_t) \
        + rho * (Pi(x_t) - u_t).dot(Pi(x_t) - u_t)
    return cost

apply_running_policy_cost(prog, Pi, mycost)

Step 2) Supervised learning on (x, u) pairs, using Augmented Lagrangian

### 3) Interactive Control

Step 1) dircol traj. optimization with L2 augmentation and L2 trajectory distance penalty.

In [30]:
from traj.dircol import make_real_dircol_mp

dircol, tree = make_real_dircol_mp(expmt="cartpole", seed=1776)

R = 10  # Cost on input "effort".
u = dircol.input()
dircol.AddRunningCost(R*u[0]**2)

Step 2) Simple supervised learning on (x, u) pairs, with L2 augmentation and L2 policy distance penalty.

In [ ]:
############################################################
# Run this cell to run the code!
############################################################

ret = prog.Solve()

x_trajectory = prog.ReconstructStateTrajectory()
breaks = np.linspace(x_trajectory.start_time(),x_trajectory.end_time(),100)
x_knots = np.hstack([x_trajectory.value(t) for t in breaks])

plt.plot(breaks, x_knots[0,:], breaks, x_knots[1,:])